# Data Automation Intern Case Study 

Important information before running this notebook : 

1. To run this notebook, you need the following python libraries (with a recent Python version, I am using Python 3.13.5) : pandas, openrouteservice, folium.
2. This notebook uses openrouteservice's free API. For this project to run properly, please create a free openrouteservice account (https://openrouteservice.org/) and add you personal API key in the next cell. 
3. This notebook is using an API, which means the code is not "working whatever the number of times we run the program". However, I deemed the API's limitations reasonable so I decided to stick to this method (40 reqests/min, and 2000/day). To overcome these limitations, I wanted to use the BlaBlaCar Bus GTFS datasets, but unfortunately the shapes table is empty. 

In [ ]:
my_key = 'please replace with your own API key'

## 1. Python Exercice

Please run the following cell to input the stations names and coordinates.

In [ ]:
def generate_stations_list() -> list:
    """
    This function prompts the user to input station names and their coordinates,
    and generates a list of tuples containing these information.

    The list has the form: [(station_name, latitude, longitude), ...]
    """

    stations_list = []

    while True:
        station_name = input("Enter the name of the station: ")

        while True:
            try:
                coordinates = input("Please enter the coordinates of the station (latitude, longitude): ")
                coordinates = coordinates.replace('(', '').replace(')', '')
                lat_str, lon_str = coordinates.split(',')
                lat = float(lat_str.strip())
                lon = float(lon_str.strip())

            except ValueError:
                print("Invalid coordinates format. Please enter them as (latitude, longitude).")
                continue

            if not (-90 <= lat <= 90):
                print("Latitude must be between -90 and 90.")
                continue
            if not (-180 <= lon <= 180):
                print("Longitude must be between -180 and 180.")
                continue

            break
            
        stations_list.append((station_name, lat, lon))
        
        another = input("Do you want to add another station? (yes/no): ").strip().lower()
        if another != 'yes':
            break

    return stations_list

stations_list = generate_stations_list()

If you want to manually add a list of stations [(station1, lat1, lon1), (station2, lat2, lon2), ...], please fill and uncomment the following line:

In [ ]:
# stations_list = [('station1', lat1, lon1), ('station2', lat2, lon2), ...]

If you want a preview of the itinerary, please run the following cell.

In [ ]:
import pandas as pd

stations_table = pd.DataFrame(stations_list, columns=['station_name', 'lat', 'lon'])
stations_table.head(10) # change the number to see more rows

The function to plot the itinerary on a map is defined below.

In [ ]:
from openrouteservice.exceptions import ApiError
import openrouteservice
import folium

def print_itinerary_map(stations_list, api_key) -> folium.Map:
    """
    This function generates a map displaying the stations and routes between them, when
    possible, in the order they are provided in the list.
    """

    client = openrouteservice.Client(key=api_key)

    # Initialize the map (zoom on first station)
    first_lat, first_lon = stations_list[0][1], stations_list[0][2]
    m = folium.Map(location=[first_lat, first_lon], zoom_start=7)

    # Display all stations on the map
    for name, lat, lon in stations_list:
        folium.Marker(
            location=[lat, lon],
            popup=name, # Display station name on click
            icon=folium.Icon(color='blue', icon='info-sign')
        ).add_to(m)

        # display the station name permanently on the map, as in the instructions example
        folium.Marker(
            location=[lat, lon],
            icon=folium.DivIcon(html=f""" 
                <div style="
                    display: inline-block;
                    background-color: white;
                    color: black;
                    padding: 2px 6px;
                    border: 1px solid black;
                    border-radius: 3px;
                    white-space: nowrap;
                    font-size: 12px;
                    box-shadow: 1px 1px 2px rgba(0,0,0,0.3);
                ">
                    {name}
                </div>
            """)
        ).add_to(m)

    # Find the first valid station (usable as route origin)
    valid_stations = []
    start_index = None

    for i, (name, lat, lon) in enumerate(stations_list):
        try:
            # Check validity by requesting a route to itself
            coords = ((lon, lat), (lon, lat))
            client.directions(coords, profile='driving-car', format='geojson')
            valid_stations.append(stations_list[i])
            start_index = i
            print(f"First valid station found: {name}")
            break
        except:
            print(f"Station \"{name}\" is not a valid starting point. Trying next...")

    if start_index is None:
        print("No valid station found. Ending script.")

    else:
        # Plot routes between valid stations starting from the first valid one
        for j in range(start_index + 1, len(stations_list)):
            start = valid_stations[-1]
            candidate = stations_list[j]
            coords = ((start[2], start[1]), (candidate[2], candidate[1]))  # lon, lat

            try:
                route = client.directions(coords, profile='driving-car', format='geojson')
                folium.GeoJson(route, name=f"Route {start[0]} → {candidate[0]}").add_to(m)
                valid_stations.append(candidate)
            except ApiError:
                print(f"Route from {start[0]} to {candidate[0]} not available. Skipping.")
            except Exception as e:
                print(f"Unexpected error between {start[0]} and {candidate[0]}: {e}")

    return m

In [ ]:
m = print_itinerary_map(stations_list, my_key)
m

# If you encounter issues with the map not displaying, you can save it to an HTML file, and open it in a web browser.
# m.save('itinerary_map.html')

## 2. Tests

Test n°1 : one of the intermediate stations is not accessible.

In [ ]:
stations_list = [
    ('Bordeaux', 44.837789, -0.579180),
    ('Nantes', 47.218371, -1.553621),
    ('Le Mans', 48.000000, 0.200000),
    ('Inccorrect Station', 50.205418, 0.989597),  # This station is not accessible
    ('Paris', 48.856613, 2.352222),
    ('Reims', 49.258329, 4.031696),
    ('Bruxelles', 50.850346, 4.351721)
]

print_itinerary_map(stations_list, my_key)

Test n°2 : The first stations are incorrect stations.

In [ ]:

stations_list = [
    ('Moutains', 46.604703,8.140242),
    ('Desert', 20, 12),
    ('Bordeaux', 44.837789, -0.579180),
    ('Atlantic ocean', 21.388334, -44.353513),
    ('Nantes', 47.218371, -1.553621),
    ('Le Mans', 48.000000, 0.200000),
    ('Paris', 48.856613, 2.352222),
    ('Reims', 49.258329, 4.031696),
    ('Bruxelles', 50.850346, 4.351721)
]

print_itinerary_map(stations_list, my_key)

Test n°3 : The final station is incorrect.

In [ ]:
stations_list = [
    ('Bordeaux', 44.837789, -0.579180),
    ('Nantes', 47.218371, -1.553621),
    ('Le Mans', 48.000000, 0.200000),
    ('Paris', 48.856613, 2.352222),
    ('Reims', 49.258329, 4.031696),
    ('Bruxelles', 50.850346, 4.351721),
    ('Inccorrect Station', 50.205418, 0.989597) # This station is not accessible
]

print_itinerary_map(stations_list, my_key)